In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from random import choice
pd.options.mode.chained_assignment = None 
import configparser
import pickle
import psycopg2
import psycopg2.extras
import contextily as ctx

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Insert-Generic-Name-Here'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
# from lonelyboy.geospatial import group_patterns as gsgp


# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')

style.use('ggplot')

get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

In [3]:
PLT_IMAGE_WIDTH = 3.748
PLT_IMAGE_HEIGHT = PLT_IMAGE_WIDTH/1.618

plt.rc('text', usetex=True)
plt.rc('font', family='sans-serif', size=8)
plt.rcParams['figure.figsize'] = (PLT_IMAGE_WIDTH, PLT_IMAGE_HEIGHT)

In [4]:
def tuple_str_to_tuple(_str_):
    return tuple(map(int, _str_[1:-1].split(',')))

def tuple_str_to_list(_str_):
    return list(map(int, _str_[1:-1].split(',')))

# Reading Data (mode: Flocks, card:5, dt:15, dist:1852)

In [20]:
df = pd.read_csv('./data/csv/GROUP_PATTERNS/convoys_card_5_dt_15_dist_1852.csv')
print(len(df))
df.head()

4749


,clusters,st,et
0,"(227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:16:00,2015-10-01 21:50:00
1,"(227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:34:00,2015-10-01 21:50:00
2,"(227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:34:00,2015-10-01 22:10:00
3,"(227612860, 228186700, 228728000, 228791000, 9...",2015-10-02 06:29:00,2015-10-02 06:49:00
4,"(205204000, 227574020, 228186700, 228728000, 2...",2015-10-02 07:49:00,2015-10-02 08:30:00


In [21]:
df.loc[:,'clusters'] = df.clusters.apply(tuple_str_to_list)
df.head()

,clusters,st,et
0,"[227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:16:00,2015-10-01 21:50:00
1,"[227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:34:00,2015-10-01 21:50:00
2,"[227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:34:00,2015-10-01 22:10:00
3,"[227612860, 228186700, 228728000, 228791000, 9...",2015-10-02 06:29:00,2015-10-02 06:49:00
4,"[205204000, 227574020, 228186700, 228728000, 2...",2015-10-02 07:49:00,2015-10-02 08:30:00


# **#4** Trip Contribution per Group Pattern (Flocks)

In [22]:
%%time
host    = 'snf-863583.vm.okeanos.grnet.gr'
db_name = 'zenodo'
uname   = 'students'
pw      = 'infol@bdbl@bs2017'
port    = '46132'

con = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port=port)
query = 'SELECT * FROM ais_data.dynamic_ships_min_trip_card_3_segmented_12h_resampled_1min_v2 WHERE mmsi IN %s AND datetime BETWEEN \'%s\' AND \'%s\';'

CPU times: user 3.02 ms, sys: 9.36 ms, total: 12.4 ms
Wall time: 3.17 s


In [23]:
df

,clusters,st,et
0,"[227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:16:00,2015-10-01 21:50:00
1,"[227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:34:00,2015-10-01 21:50:00
2,"[227142200, 227319570, 228160000, 228213700, 2...",2015-10-01 21:34:00,2015-10-01 22:10:00
3,"[227612860, 228186700, 228728000, 228791000, 9...",2015-10-02 06:29:00,2015-10-02 06:49:00
4,"[205204000, 227574020, 228186700, 228728000, 2...",2015-10-02 07:49:00,2015-10-02 08:30:00
5,"[205204000, 227574020, 228186700, 228728000, 2...",2015-10-02 07:51:00,2015-10-02 08:30:00
6,"[205204000, 228186700, 228728000, 228791000, 2...",2015-10-02 07:51:00,2015-10-02 08:39:00
7,"[205204000, 228186700, 228791000, 275346000, 2...",2015-10-02 08:53:00,2015-10-02 10:38:00
8,"[205204000, 228186700, 228791000, 259019000, 2...",2015-10-02 08:57:00,2015-10-02 10:38:00
9,"[205204000, 227394750, 228064900, 228186700, 2...",2015-10-02 11:31:00,2015-10-02 11:46:00


In [24]:
ports = pd.read_pickle('data/pkl/ports_raw.pkl')
ports = gspp.create_port_bounds(ports, port_radius=2000, epsg=2154)

def points_within_geometry(df, geometry):
    sindex = df.sindex # create the spatial index (r-tree) of the vessel's data points     
    # find the points that intersect with each subpolygon and add them to _points_within_geometry_ DataFrame
    points_within_geometry = pd.DataFrame()
    for poly in geometry.geom:
        # find approximate matches with r-tree, then precise matches from those approximate ones
        possible_matches_index = list(sindex.intersection(poly.bounds))
        possible_matches = df.iloc[possible_matches_index]
        precise_matches = possible_matches[possible_matches.intersects(poly)]
        points_within_geometry = points_within_geometry.append(precise_matches)

    points_within_geometry = points_within_geometry.drop_duplicates(subset=['mmsi', 'ts'])
    points_outside_geometry = df[~df.isin(points_within_geometry)].dropna(how='all')

    df.loc[:,'port_label'] = np.nan
    # When we create the _traj_id_ column, we label each record with 0, 
    # if it's outside the port's radius and -1 if it's inside the port's radius. 
    df.loc[df.index.isin(points_within_geometry.index), 'port_label'] = -1
    df.loc[df.index.isin(points_outside_geometry.index), 'port_label'] = 0
    return df

In [26]:
import multiprocessing
from tqdm import tqdm_notebook

def classify_trips_v2(df):        
    if df['port_label'].iloc[0] == -1 and df['port_label'].iloc[-1] == -1:
        return np.array([1])
    elif df['port_label'].iloc[0] == -1 and df['port_label'].iloc[-1] == 0:
        return np.array([2])
    elif df['port_label'].iloc[0] == 0 and df['port_label'].iloc[-1] == -1:
        return np.array([3])
    elif df['port_label'].iloc[0] == 0 and df['port_label'].iloc[-1] == 0:
        return np.array([4])
    else:
        print ("$H1T!!!")


def get_gp_trip_contributions_parallel(y):
    print ('Connecting to Database...')
    con = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port=port)
    print ('Connected to Database!')
    
    df_stat4 = []
    for row in tqdm_notebook(y.itertuples(), total=len(y)):
        df_stat4_row = pd.DataFrame([{'GP':row.clusters, 'C1':0, 'C2':0, 'C3':0, 'C4':0}], columns=['GP', 'C1', 'C2', 'C3', 'C4'])

        row_dynamic = pd.read_sql_query(query%(tuple(row.clusters), row.st, row.et), con=con)
#         row_dynamic = gspp.gdf_from_df(row_dynamic)
#         row_dynamic = points_within_geometry(row_dynamic, ports)
        
        row_dynamic_trips = row_dynamic.groupby(['mmsi', 'trip_id']).apply(classify_trips_v2).to_frame()
        trip_contr = row_dynamic_trips[0].value_counts().dropna().index.values
        df_stat4_row.iloc[:, trip_contr[0][0]] = 1

        df_stat4.append(df_stat4_row)
    con.close()
    return pd.concat(df_stat4)


def parallelize_dataframe(df_par, func):
    num_cores = multiprocessing.cpu_count()-1  #leave one free to not freeze machine
    num_partitions = num_cores #number of partitions to split dataframe
    
    df_split = np.array_split(df_par, num_partitions)
    pool = multiprocessing.Pool(num_cores)
    
    df_res = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df_res

In [27]:
df_tc = parallelize_dataframe(df, get_gp_trip_contributions_parallel)

Connecting to Database...
Connecting to Database...
Connecting to Database...
Connecting to Database...
Connecting to Database...
Connecting to Database...
Connecting to Database...
Connected to Database!
Connected to Database!
Connected to Database!
Connected to Database!
Connected to Database!
Connected to Database!
Connected to Database!


* # For Flocks (vessel-independent)

In [18]:
df_tc.iloc[:,1:].sum()

C1        1
C2       38
C3       10
C4    34242
dtype: int64

In [19]:
df_tc.to_csv('./flocks_card_5_dt_15_dist_1852_trip_contributions_V2.csv', header=True, index=False)

* # For Convoys (vessel-independent)

In [30]:
df_tc.iloc[:,1:].sum()

C1       8
C2      52
C3      31
C4    4658
dtype: int64

In [31]:
df_tc.to_csv('./convoys_card_5_dt_15_dist_1852_trip_contributions_V2.csv', header=True, index=False)